## Prepare environment

In [1]:
import pandas as pd
from datetime import timedelta
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn import svm
from sklearn import metrics
import bokeh.plotting as bp
import itertools
from sklearn.model_selection import GridSearchCV
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.tools.plotting import autocorrelation_plot
import datetime
from fbprophet import Prophet
from sklearn.preprocessing import MinMaxScaler

## Acquire

In [2]:
df = pd.read_csv('south_central_moisture_data_1988_2017.csv')
df.head()

,final_date,soil_moisture,pdsi
0,1988-03-01,4.79,0.0
1,1988-03-08,4.67,0.0
2,1988-03-15,4.24,0.0
3,1988-03-22,5.59,0.0
4,1988-03-29,5.00,0.0


## Scale data

In [3]:
df = df[['final_date', 'soil_moisture']]

mmscaler = MinMaxScaler(feature_range=(0, 1))
mmscaler.fit(df[['soil_moisture']])
df.soil_moisture = mmscaler.transform(df[['soil_moisture']])

df.head()

,final_date,soil_moisture
0,1988-03-01,0.598248
1,1988-03-08,0.583229
2,1988-03-15,0.529412
3,1988-03-22,0.698373
4,1988-03-29,0.624531


## Set index

In [4]:
df.final_date = pd.to_datetime(df.final_date)
df = df.set_index('final_date')

In [5]:
df_agg = df.soil_moisture.resample('W').agg('median')

In [6]:
df_agg.head()

final_date
1988-03-06    0.598248
1988-03-13    0.583229
1988-03-20    0.529412
1988-03-27    0.698373
1988-04-03    0.624531
Freq: W-SUN, Name: soil_moisture, dtype: float64

## Only get data from 1997-present to match PDSI data

In [7]:
df_agg = df_agg['1997':]

In [8]:
df_agg = df_agg.to_frame(name=None)

In [9]:
df_agg.head()

,soil_moisture
final_date,
1997-01-05,0.446809
1997-01-12,0.515645
1997-01-19,0.528160
1997-01-26,0.658323
1997-02-02,0.639549


Compute soil moisture mean from weekly aggregate medians.

In [10]:
agg_mean = df_agg.soil_moisture.mean()
agg_mean

0.4830032772573354

Computer soil moisture standard deviation from weekly aggregate medians.

In [11]:
agg_stdev = df_agg.soil_moisture.std()
agg_stdev

0.35018915819138896

Compute mean-(3xstdev) to get lower bound

In [12]:
df_agg["lower_bound"] = (agg_mean - (4*agg_stdev))

In [13]:
df_agg.head()

,soil_moisture,lower_bound
final_date,,
1997-01-05,0.446809,-0.917753
1997-01-12,0.515645,-0.917753
1997-01-19,0.528160,-0.917753
1997-01-26,0.658323,-0.917753
1997-02-02,0.639549,-0.917753


In [14]:
df_agg.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1105 entries, 1997-01-05 to 2018-03-04
Freq: W-SUN
Data columns (total 2 columns):
soil_moisture    1101 non-null float64
lower_bound      1105 non-null float64
dtypes: float64(2)
memory usage: 25.9 KB


There are four weeks missing data. These will be backfilled.

In [15]:
df_agg[df_agg.soil_moisture.isnull()]

,soil_moisture,lower_bound
final_date,,
1999-02-28,NaN,-0.917753
2004-02-29,NaN,-0.917753
2010-02-28,NaN,-0.917753
2016-02-28,NaN,-0.917753


In [16]:
df_agg.soil_moisture.fillna(method='bfill', inplace=True)

In [17]:
df_agg.soil_moisture.isnull().sum()

0

Created a column for "soil moisture - lower bound"

In [18]:
df_agg["sm_lb"] = (df_agg.soil_moisture - df_agg.lower_bound)

In [19]:
df_agg.head()

,soil_moisture,lower_bound,sm_lb
final_date,,,
1997-01-05,0.446809,-0.917753,1.364562
1997-01-12,0.515645,-0.917753,1.433398
1997-01-19,0.528160,-0.917753,1.445914
1997-01-26,0.658323,-0.917753,1.576076
1997-02-02,0.639549,-0.917753,1.557303


Created a boolean column for whether or not this value falls below normal range

In [20]:
df_agg['outside_normal'] = (df_agg.sm_lb < 1)

In [21]:
df_agg.head()

,soil_moisture,lower_bound,sm_lb,outside_normal
final_date,,,,
1997-01-05,0.446809,-0.917753,1.364562,False
1997-01-12,0.515645,-0.917753,1.433398,False
1997-01-19,0.528160,-0.917753,1.445914,False
1997-01-26,0.658323,-0.917753,1.576076,False
1997-02-02,0.639549,-0.917753,1.557303,False


In [22]:
df_agg.outside_normal.sum()

195

In [23]:
df_agg.soil_moisture.count()

1105

In [25]:
(195/1105)*100

17.647058823529413

In [26]:
df_agg[df_agg.sm_lb < 1]

,soil_moisture,lower_bound,sm_lb,outside_normal
final_date,,,,
1998-07-19,0.080100,-0.917753,0.997853,True
1998-07-26,0.062578,-0.917753,0.980332,True
1998-08-02,0.048811,-0.917753,0.966564,True
1998-08-09,0.046308,-0.917753,0.964061,True
1998-08-16,0.040050,-0.917753,0.957803,True
1998-09-06,0.043805,-0.917753,0.961558,True
1999-09-26,0.076345,-0.917753,0.994099,True
1999-10-03,0.066333,-0.917753,0.984086,True
1999-10-10,0.063830,-0.917753,0.981583,True
